In [1]:
cg_list = [5, 15, 25,35]

In [2]:
cg_list = [10, 19, 50]

In [2]:
for ncgs in cg_list:
    dataset = "sarscov2"
    steps = 3
    coarse_grained_model_type = "CalphaBasedModel" #@param ["CalphaBasedModel", "ResidueBasedModel", "SidechainModel", "CalphaCMModel", "CalphaSCModel", "BackboneModel", "MainchainModel", "Martini", "Martini3", "PRIMO"]

    input_pdb = f"/home/erin/Documents/cg2all/{steps}step/{dataset}/ncg{ncgs}_recon.pdb"
    input_dcd_path = f"/home/erin/Documents/cg2all/{steps}step/{dataset}/ncg{ncgs}_recon.dcd"
    !mdconvert $input_pdb -o $input_dcd_path -f
    #input_pdb = list(input_pdb)[0]
    #input_dcd = files.upload()
    input_dcd = f"/home/erin/Documents/cg2all/{steps}step/{dataset}/ncg{ncgs}_recon.dcd"
    output_pdb_recon = f"/home/erin/Documents/cg2all/{steps}step/{dataset}/ncg{ncgs}_FG_recon.pdb"
    output_pdb_clean = f"/home/erin/Documents/cg2all/{steps}step/{dataset}/ncg{ncgs}_FG_recon_clean.pdb"

    batch_size = 5 #@param {type: "number"}
    #@markdown - Batch size should be a divisor of the total number of frames. The example trajectory file has five frames.

    # convert
    import pathlib
    output_dcd = pathlib.Path(input_dcd).stem + ".all.dcd"
    output_pdb = pathlib.Path(input_pdb).stem + ".all.pdb"

    !convert_cg2all -p $input_pdb --dcd $input_dcd -o $output_dcd -opdb $output_pdb --cg $coarse_grained_model_type --batch $batch_size
    print(f"Converted {input_dcd} in {coarse_grained_model_type} to {output_dcd}")

    # import mdtraj
    # pdb = mdtraj.load(input_pdb)
    # traj = mdtraj.load(output_dcd, top=output_pdb)
    # if len(traj) > 20:
    #     print("Displaying first 20 frames")
    #     traj = traj[:20]
    # traj = traj.superpose(traj)
    #traj.save("display.pdb")
    #
    #display("display.pdb", representation="cartoon", is_traj=True).show()
    #print("Displaying the last frame")
    #display(output_pdb, representation="cartoon").show()
    !mdconvert $output_dcd -t $output_pdb -o $output_pdb_recon -f
    !python /home/erin/Documents/cg2all/postprocessing.py --action clean --input $output_pdb_recon
    if ncgs ==5:
        !python /home/erin/Documents/cg2all/postprocessing.py --action clean --input /home/erin/Documents/cg2all/3step/sarscov2/FG_ground_truth.pdb
    !python /home/erin/Documents/cg2all/postprocessing.py --action rmsd --recon $output_pdb_clean --truth /home/erin/Documents/cg2all/2step/sarscov2/FG_ground_truth_clean.pdb

/home/erin/miniconda3/envs/cg2all/lib/python3.8/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting xyz dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),
converted 300 frames, 320 atoms 
100%|███████████████████████████████████████████| 60/60 [00:25<00:00,  2.37it/s]
{'loading_ckpt': 0.600285530090332, 'model_configuration': 0.03897213935852051, 'loading_input': 2.8542726039886475, 'forward_pass': 24.85145092010498, 'writing_output': 0.21510744094848633, 'total': 28.560088634490967}
Converted /home/erin/Documents/cg2all/3step/sarscov2/ncg25_recon.dcd in CalphaBasedModel to ncg25_recon.all.dcd
converted 300 frames, 5035 atoms 
0.33434075
